In [1]:
print("Shivaya Namah")

Shivaya Namah


In [7]:
name: str = "Hi"
name


'Hi'

In [24]:
from langchain_cohere.chat_models import ChatCohere
from pydantic_settings import BaseSettings
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from langchain_ibm.chat_models import ChatWatsonx


class Settings(BaseSettings): 

    pinecone_api_key: str 
    watsonx_apikey: str  
    watsonx_url: str
    watsonx_project_id: str
    cohere_api_key: str 
    tavily_api_key: str
    google_api_key: str

    langsmith_api_key: str
    langsmith_endpoint: str
    langsmith_tracing: bool
    langsmith_project: str

    class Config:
        env_file = ".env"
        extra = "ignore"

settings = Settings()
print("API Keys loaded succesfully.")

API Keys loaded succesfully.


In [25]:
class Country(BaseModel):

    "Represents the Country"
    
    name: str = Field(description = "Name of the country.")
    language: str = Field(description = "National Language of the country.")
    capital: str = Field(description = "Captital city of the country.")


In [30]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", api_key = settings.google_api_key, temperature = 0)
llm

ChatGoogleGenerativeAI(model='models/gemini-2.5-pro', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001FA0DC817F0>, default_metadata=(), model_kwargs={})

In [27]:
llm = ChatWatsonx(
    apikey = settings.watsonx_apikey,
    project_id = settings.watsonx_project_id,
    url = settings.watsonx_url,
    temperature = 0.7,
    model_id = "ibm/granite-3-3-8b-instruct"
)
llm

ChatWatsonx(model_id='ibm/granite-3-3-8b-instruct', project_id='25fa768b-a90b-433c-b4a4-66bb407af16b', url=SecretStr('**********'), apikey=SecretStr('**********'), params={'temperature': 0.7}, temperature=0.7, watsonx_model=<ibm_watsonx_ai.foundation_models.inference.model_inference.ModelInference object at 0x000001FA0C1ADA70>)

In [31]:
structured_llm = llm.with_structured_output(schema = Country)
structured_llm

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-2.5-pro', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001FA0DC817F0>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Country', 'description': 'Represents the Country', 'parameters': {'properties': {'name': {'description': 'Name of the country.', 'type': 'string'}, 'language': {'description': 'National Language of the country.', 'type': 'string'}, 'capital': {'description': 'Captital city of the country.', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'type': 'object'}}}], 'ls_structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'name': 'Country', 'description': 'Represents the Country', 'parameters': {'properties': {'name': {'description': 'Name of the country.'

In [32]:
response = structured_llm.invoke("Tell me about India")

In [ ]:
response

Country(name='India', language='Hindi', capital='New Delhi')

In [ ]:
structured_llm.invoke("Tell me about France")

Country(name='France', language='French', capital='Paris')

- The above perform Validation after llm generates the output. 
- If you don't want that validation we simply go with TypedDict, dict something

In [10]:
from typing_extensions import TypedDict, Annotated

class Country2(TypedDict):

    name: Annotated[str, ..., "Name of the country"]
    language: Annotated[str, ...,  "Language of the country"]
    capital: Annotated[str, ...,  "Capital City of the country"]


In [11]:
structured_llm = llm.with_structured_output(Country2)
structured_llm

RunnableBinding(bound=ChatWatsonx(model_id='ibm/granite-3-3-8b-instruct', project_id='25fa768b-a90b-433c-b4a4-66bb407af16b', url=SecretStr('**********'), apikey=SecretStr('**********'), params={'temperature': 0.7}, temperature=0.7, watsonx_model=<ibm_watsonx_ai.foundation_models.inference.model_inference.ModelInference object at 0x000001FA0C15E0D0>), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Country2', 'description': "dict() -> new empty dictionary\ndict(mapping) -> new dictionary initialized from a mapping object's\n    (key, value) pairs\ndict(iterable) -> new dictionary initialized as if via:\n    d = {}\n    for k, v in iterable:\n        d[k] = v\ndict(**kwargs) -> new dictionary initialized with the name=value pairs\n    in the keyword argument list.  For example:  dict(one=1, two=2)", 'parameters': {'type': 'object', 'properties': {'name': {'description': 'Name of the country', 'type': 'string'}, 'language': {'description': 'Language of the country', 'type': '

In [12]:
structured_llm.invoke("Tell me about France")

{'capital': 'Paris', 'language': 'French', 'name': 'France'}

In [14]:
structured_llm.invoke("Tell me about China")

{'capital': 'Beijing', 'language': 'Mandarin', 'name': 'China'}

In [15]:
structured_llm.invoke("Tell me about India")

{'capital': 'New Delhi', 'language': 'Hindi', 'name': 'India'}

#### This is the problem, sometimes llm wont  return the actual output thats why we use, Pydantic